In [107]:
# install packages
%pip install pandas
%pip install plotly


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [108]:
# import packages
import math

### ASHRAE 55:2023 Method for Calculating PMV

In [109]:
def calculate_clo(m: float, icl: float) -> float:
    """
    Calculates the clo for a given metabolic rate and clothing insulation using ASHRAE 55:2023.
    
    Args:
        m (float): metabolic rate in met unit
        icl (float): clothing insulation in clo unit
    
    Return:
        float: clo value to use in the PMV calculator
    """
    if m < 1.2:
        clo = icl

    else:
        clo = icl * (0.6 + 0.4 / m)

    return clo

In [ ]:
def calculate_vel(vav: float, vag:float, met: float) -> float:
    """
    Calculate the air speed (m/s).

    Args:
        vav (float): average air velocity in m/s
        vag (float): activity generated air velocity in m/s
        met (float): metabolic rate in met

    Return:
        float: air speed in m/s
    """
    vag = 0.3 * (met - 1) if met > 1 else 0
    vel = vag + vav
    return vel

In [110]:
def calculate_pmv(ta: float, tr: float, vel: float, rh: float, met: float, clo: float, wme: float) -> dict:
     """
     Calculate the PMV for given inputs using ASHRAE 55:2023. The air speed shouldn't be above 0.1 m/s.

     Args:
          ta (float): air temperature in C
          tr (float): mean radiant temperature in C
          vel(float): air speed in m/s
          rh (float): relative humidity in %
          met (float): metabolic rate in met unit
          clo (float): clothing factor in clo unit (calculated using calculate_clo method)
          wme (float): external work in met

     Return:
          dict: {
               pmv: Predicted Mean Vote (PMV), 
               ppd: Predicted Precentage of Dissatisfied (PPD),
               status: status of the calculation, 0 succeed, 1 error
               }
     """
     if met > 1:
          pa = rh * 10. * math.exp(16.6536 - 4030.183 / (ta + 235.))
          icl = 0.155 * clo # thermal insulation of the clothing in m2K/W
          m = met * 58.15 # metabolic rate in W/m2
          w = wme * 58.15 # extenal work in W/m2
          mw = m - w # internal heat production in the humen body

          if icl <= 0.078:
               fcl = 1 + (1.29 * icl)
          else:
               fcl = 1.05 + (0.645 * icl)

          # heat transfer coefficient by forced convection
          hcf = 12.1 * math.sqrt(vel)
          taa = ta + 273.
          tra = tr + 273.
          tcla = taa + (35.5 - ta) / (3.5 * icl + 0.1)
          p1 = icl * fcl
          p2 = p1 * 3.96
          p3 = p1 * 100.
          p4 = p1 * taa
          p5 = 308.7 - 0.028 * mw + (p2 * math.pow(tra / 100, 4))
          xn = tcla / 100.
          xf = tcla / 50.
          eps = 0.00015
          n = 0

          while abs(xn - xf) > eps:
               xf = (xf + xn) / 2
               hcn = 2.38 * math.pow(abs(100. * xf - taa), 0.25)
               hc = hcf if hcf > hcn else hcn
               xn = (p5 + p4 * hc - p2 * math.pow(xf, 4)) / (100. + p3 * hc)
               n += 1
               if n > 150:
                    print ("Max iterations exceeded")
                    return {
                         'pmv': 0,
                         'ppd': 0,
                         'status': 1
                         }

          tcl = 100. * xn - 273.
          
          # heat loss diff. through skin
          hl1 = 3.05 * 0.001 * (5733 - (6.99 * mw) - pa)

          # heat loss by sweating
          hl2 = 0.42 * (mw - 58.15) if (mw > 58.15) else 0

          # latent respiration heat loss
          hl3 = 1.7 * 0.00001 * m * (5867. - pa)
          
          # dry respiration heat loss
          hl4 = 0.0014 * m * (34. - ta)
          
          # heat loss by radiation
          hl5 = 3.96 * fcl * (math.pow(xn, 4) - math.pow(tra / 100., 4))

          # heat loss by convection
          hl6 = fcl * hc * (tcl - ta)

          ts = 0.303 * math.exp(-0.036 * m) + 0.028
          pmv = ts * (mw - hl1 - hl2 - hl3 - hl4 - hl5 - hl6)
          ppd = 100 - 95 * math.exp(-0.03353 * math.pow(pmv, 4) - 0.2179 * math.pow(pmv, 2))

          return {
               'pmv': pmv,
               'ppd': ppd,
               'status': 0
               }

     else:
          print("ASHRAE 55:2023 calculation does not available for metabolic rate less than 1")
          return {
              'pmv': 0,
              'ppd': 0,
              'status': 1
              }
     
     

### Method Validation

In [111]:
validation_list = [
    {'ta': 19.6, 'rh': 86, 'tr': 19.6, 'vel': 0.1, 'met': 1.1, 'clo': 1, 'pmv': -0.47}, #1
    {'ta': 23.9, 'rh': 66, 'tr': 23.9, 'vel': 0.1, 'met': 1.1, 'clo': 1, 'pmv': 0.48}, #2
    {'ta': 25.7, 'rh': 15, 'tr': 25.7, 'vel': 0.1, 'met': 1.1, 'clo': 1, 'pmv': 0.53}, #3
    {'ta': 21.2, 'rh': 20, 'tr': 21.2, 'vel': 0.1, 'met': 1.1, 'clo': 1, 'pmv': -0.48}, #4
    {'ta': 23.6, 'rh': 67, 'tr': 23.6, 'vel': 0.1, 'met': 1.1, 'clo': 0.5, 'pmv': -0.47}, #5
    {'ta': 26.8, 'rh': 56, 'tr': 26.8, 'vel': 0.1, 'met': 1.1, 'clo': 0.5, 'pmv': 0.52}, #6
    {'ta': 27.9, 'rh': 13, 'tr': 27.9, 'vel': 0.1, 'met': 1.1, 'clo': 0.5, 'pmv': 0.5}, #7
    {'ta': 24.7, 'rh': 16, 'tr': 24.7, 'vel': 0.1, 'met': 1.1, 'clo': 0.5, 'pmv': -0.49} #8
]

In [112]:
def test_pmv_calculation(ta: float, tr: float, vel: float, rh: float, met: float, clo: float, wme: float, test_pmv: float) -> dict:
    """
    Test the PMV calculation method defined in the previous steps.

     Args:
          ta (float): air temperature in C
          tr (float): mean radiant temperature in C
          vel(float): air speed in m/s
          rh (float): relative humidity in %
          met (float): metabolic rate in met unit
          clo (float): clothing factor in clo unit (calculated using calculate_clo method)
          wme (float): external work in met
          test_pmv (float): PMV value given in the validation table

     Return:
          dict: {
               calculated_pmv: PMV value calculated using the cclculate_pmv method, 
               test_pmv: PMV given in the validation table,
               deviation: prcentage deviation from the validation PMV
               result: "PASS" or "FAIL" based on the deviation value, less than 10% concidered as "PASS"
               status: status of the calculation, 0 succeed, 1 error
               }
    """
    pmv_result = calculate_pmv(ta=ta, tr=tr, vel=vel, rh=rh, met=met, clo=clo, wme=wme)

    if pmv_result['status'] == 0:
        pmv = pmv_result['pmv']
        deviation = (pmv - test_pmv) / test_pmv * 100.
        result = "PASS" if deviation < 10 else "FAIL"
        return {
            "calculated_pmv": pmv, 
            "test_pmv": test_pmv, 
            "deviation": deviation, 
            "result": result, 
            "status": pmv_result['status']
            }
    
    else:
        return {
            "calculated_pmv": None, 
            "test_pmv": None, 
            "deviation": None, 
            "result": None, 
            "status": pmv_result['status']
            }

In [113]:
for n, item in enumerate(validation_list):
    test_result = test_pmv_calculation(ta = item['ta'], tr = item['tr'], vel = item['vel'], rh = item['rh'], met = item['met'], clo = item['clo'], wme = 0, test_pmv = item['pmv'])
    print(f'Test Number: {n}\n Calculated PMV: {test_result['calculated_pmv']}\n Given PMV: {test_result['test_pmv']}\n Deviation: {test_result['deviation']}%\n Result: {test_result['result']}')

Test Number: 0
 Calculated PMV: -0.47467198128887644
 Given PMV: -0.47
 Deviation: 0.9940385721013767%
 Result: PASS
Test Number: 1
 Calculated PMV: 0.47727708611081504
 Given PMV: 0.48
 Deviation: -0.5672737269135295%
 Result: PASS
Test Number: 2
 Calculated PMV: 0.5275205662501071
 Given PMV: 0.53
 Deviation: -0.46781768865903783%
 Result: PASS
Test Number: 3
 Calculated PMV: -0.4750504818720359
 Given PMV: -0.48
 Deviation: -1.0311496099925146%
 Result: PASS
Test Number: 4
 Calculated PMV: -0.4721506758411812
 Given PMV: -0.47
 Deviation: 0.45759060450664396%
 Result: PASS
Test Number: 5
 Calculated PMV: 0.5170046862594345
 Given PMV: 0.52
 Deviation: -0.5760218731856726%
 Result: PASS
Test Number: 6
 Calculated PMV: 0.5002402909067151
 Given PMV: 0.5
 Deviation: 0.04805818134301365%
 Result: PASS
Test Number: 7
 Calculated PMV: -0.4860260068301159
 Given PMV: -0.49
 Deviation: -0.8110190142620645%
 Result: PASS
